In [1]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
import aplpy
import glob
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from astropy.io import fits
import astropy.coordinates as coord
from astropy.wcs import WCS
import montage_wrapper as m
import matplotlib.patheffects as PE
import brewer2mpl
set2 = brewer2mpl.get_map('Set2', 'qualitative', 8).mpl_colors
dark2 = brewer2mpl.get_map('Dark2', 'qualitative', 8).mpl_colors
import prettyplotlib as ppl
import matplotlib.patches as patches
import matplotlib.lines as lines
from matplotlib import rc
from cycler import cycler
rc('font', **{'family': 'serif', 'serif': ['Computer Modern Roman'], 'size':14})
rc('text', usetex=True)
rc('axes', prop_cycle=cycler('color', set2))

In [2]:
filt = 'fitsimg'

In [3]:
if not os.path.exists('../calibrated_files/{}mosaic/'.format(filt)):
    m.wrappers.mosaic('../calibrated_files/{}/'.format(filt),'../calibrated_files/{}mosaic/'.format(filt))

In [4]:
fitsfile = '../calibrated_files/{}mosaic/mosaic.fits'.format(filt)
#with fits.open(fitsfile) as hdu:
wcs2 = WCS(fitsfile)
    #dat = hdu[0].data

In [5]:
blerp = pd.read_csv('Kaluzny2004rrl.txt', usecols=[0,1,2,4], delim_whitespace=True)
crowd = pd.read_csv('final_data_files/all_possible_phot_resid_braga.csv')

rej = blerp[~blerp.id.isin(crowd.id)]

rejab = rej[rej.type == 0]
rejc = rej[rej.type == 1]

typeab = blerp[blerp.type == 0]
typec = blerp[blerp.type == 1]

def get_x_y(df):
    ra1 = coord.Angle(df.ra, unit='hour')
    abra = ra1.degree
    dec1 = coord.Angle(df.dec, unit='deg')
    abdec = dec1.degree
    #x1, y1 = wcs2.all_world2pix(abra, abdec, 1)
    return abra, abdec

x1, y1 = get_x_y(typeab)
x2, y2 = get_x_y(typec)
x3, y3 = get_x_y(rejab)
x4, y4 = get_x_y(rejc)

In [ ]:
#fig, ax0 = plt.subplots(1,1,figsize=(9,8))
ax = aplpy.FITSFigure(fitsfile,north=True)
ax.show_grayscale(vmin=10**-0.5,vmax=10**1.3,stretch='log')
ax.set_theme('publication')
#ax.ticks.hide()
ax.tick_labels.set_yformat('dd:mm')
ax.tick_labels.set_xformat('hh:mm')
#ax.recenter(ra_center, dec_center, width=width/1.5, height=height)
ax.frame.set_linewidth(0.75)

# for i in glob.glob('../../ocen_images/omegaCen_*.fits'):
#     mapnum = int(i.split('_')[2])
#     filt = i.split('_')[5].split('.')[0]
#     #with fits.open(i) as f:
#     instrument_wcs = WCS(fits.getheader(i,0))
#     vertices = instrument_wcs.calc_footprint().T
#     if filt == '3p6um':
#         if mapnum == 1:
#             ax.show_polygons([vertices], linewidth = 3, facecolor='none',
#                              edgecolor='k', label = '$3.6\ \mu$m', zorder = 5-mapnum)
#         else:
#             ax.show_polygons([vertices], linewidth = 3, facecolor='none',
#                              edgecolor='k', zorder = 5-mapnum)
#     if filt == '4p5um':
#         if mapnum == 1:
#             ax.show_polygons([vertices], linewidth = 2, facecolor='none',
#                              edgecolor='w', label = '$4.5\ \mu$m', zorder = 5-mapnum,
#                              path_effects=[PE.withStroke(linewidth=3.5, foreground='k')])
#         else:
#             ax.show_polygons([vertices], linewidth = 2, facecolor='none',
#                              edgecolor='w', zorder = 5-mapnum,
#                              path_effects=[PE.withStroke(linewidth=3.5, foreground='k')])
ax.show_markers(x1, y1, s=35., alpha=1, facecolor=dark2[3], linewidth=0.5, 
                label='RRab', rasterized=True, edgecolor='k')
ax.show_markers(x2, y2, s=40., marker='^', alpha=1, facecolor=dark2[3], linewidth=0.5, 
                label='RRc', rasterized=True, edgecolor='k')
ax.show_markers(x3, y3, s=35., alpha=1, facecolor='#FAFAFA', linewidth=0.5, 
                label='RRab', rasterized=True, edgecolor='k')
ax.show_markers(x4, y4, s=40., marker='^', alpha=1, facecolor='#FAFAFA', linewidth=0.5, 
                label='RRc', rasterized=True, edgecolor='k')
# ax.set_title('IRAC (both filters)')# [{}.{}]'.format(*filt.split('p')))

# patch1 = patches.Rectangle([0, 0], 1, 1, linewidth = 3, facecolor='none',
#                            edgecolor='k')
# patch2 = patches.Rectangle([0, 0], 1, 1, linewidth = 2, facecolor='none',
#                              edgecolor='w',
#                              path_effects=[PE.withStroke(linewidth=3.5, foreground='k')])

rrab = lines.Line2D([], [], marker='o', markersize=np.sqrt(35.), alpha=1, color=dark2[3], lw=0)
rrc = lines.Line2D([], [], marker='^', markersize=np.sqrt(40.), alpha=1, color=dark2[3], lw=0)
rrab_rej = lines.Line2D([], [], marker='o', markersize=np.sqrt(35.), alpha=1, color='#FAFAFA', lw=0)
rrc_rej = lines.Line2D([], [], marker='^', markersize=np.sqrt(40.), alpha=1, color='#FAFAFA', lw=0)

#handles0, labels0 = ax.get_legend_handles_labels()
handles = [rrab, rrc]
labels = ['RRab', 'RRc']

ppl.legend(handles=handles, labels=labels, loc=1, prop={'size':14}, handlelength=1, numpoints=1)

ax.save('final_plots/coverage_both_newimages.eps', dpi=400)